In [1]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline


import jax.numpy as np
import pickle
import itertools
'unset XLA_FLAGS'

from chainconsumer import ChainConsumer
from chainconsumer.analysis import Analysis

Populating the interactive namespace from numpy and matplotlib


## Load data

In [3]:

with open("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/sample_nd_mse.pkl", 'rb') as g:
    sample_nd_mse = pickle.load(g)
with open("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/sample_nd_mae.pkl", 'rb') as g:
    sample_nd_mae= pickle.load(g)
with open("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/sample_nd_vmim.pkl", 'rb') as g:
    sample_nd_vmim = pickle.load(g)
with open("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/sample_nd_gnll.pkl", 'rb') as g:
    sample_nd_gnll = pickle.load(g)    

sample_ff=np.load("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/posterior_full_field__256N_10ms_27gpa_0.26se.npy",allow_pickle=True)

sample_ps=np.load("/gpfsdswork/projects/rech/ykz/ulm75uc/VMIM-vs-MSE-/data/posterior_power_spectrum__256N_10ms_27gpa_0.26se.npy",allow_pickle=True)


In [4]:
name_parameters= ['Omegac', 'Omegab', 'sigma8', 'h0', 'ns', 'w0']

In [5]:
sample=sample_nd_gnll
c = ChainConsumer()
c.add_chain([sample[:,i] for i in range(6)], parameters= name_parameters)

In [6]:

lis=[0,1,2,3,4,5]

f_o_m= {}
for i, j in itertools.combinations(lis, 2):
    cov=c.analysis.get_covariance(parameters=[name_parameters[i],name_parameters[j]])
    fom=sqrt(np.linalg.det(np.linalg.pinv(cov[1])))
    name=cov[0]
    f_o_m[str(name)] = fom
            

In [7]:
f_o_m

{"['Omegac', 'Omegab']": 6297.176,
 "['Omegac', 'sigma8']": 1881.8647,
 "['Omegac', 'h0']": 616.28,
 "['Omegac', 'ns']": 1035.3643,
 "['Omegac', 'w0']": 178.29556,
 "['Omegab', 'sigma8']": 4695.7285,
 "['Omegab', 'h0']": 2820.6067,
 "['Omegab', 'ns']": 4034.5547,
 "['Omegab', 'w0']": 817.9895,
 "['sigma8', 'h0']": 467.92267,
 "['sigma8', 'ns']": 692.1771,
 "['sigma8', 'w0']": 152.49493,
 "['h0', 'ns']": 425.61755,
 "['h0', 'w0']": 81.84353,
 "['ns', 'w0']": 118.26017}

In [8]:
np.sqrt(np.diag(c.analysis.get_covariance(parameters=name_parameters)[1]))

Array([0.02722519, 0.00594268, 0.03585207, 0.05977336, 0.04175357,
       0.2060798 ], dtype=float32)

In [9]:
name_parameters

['Omegac', 'Omegab', 'sigma8', 'h0', 'ns', 'w0']

In [10]:
c.analysis.get_summary()

{'Omegac': [0.24909471379842363, 0.2745548422453248, 0.3032340673924089],
 'Omegab': [0.04322887170372518, 0.049473023453457915, 0.05516709516809515],
 'sigma8': [0.7986817539850516, 0.8339398554278297, 0.8698058551713452],
 'h0': [0.6088607900879529, 0.6700395094562075, 0.7294616618204515],
 'ns': [0.9122245476901935, 0.9539020727400573, 0.9960590737418223],
 'w0': [-0.9440850711184983, -0.7287313393609107, -0.5382189313880232]}

In [11]:
c.analysis.get_latex_table( transpose=True)


'\\begin{table}\n    \\centering\n    \\caption{}\n    \\label{tab:model_params}\n    \\begin{tabular}{cc}\n        \\hline\n\t\tParameter & Chain 0 \\\\ \n\t\t\\hline\n\t\tOmegac & $0.275^{+0.029}_{-0.025}$ \\\\ \n\t\tOmegab & $\\left( 49.5^{+5.7}_{-6.2} \\right) \\times 10^{-3}$ \\\\ \n\t\tsigma8 & $0.834^{+0.036}_{-0.035}$ \\\\ \n\t\th0 & $0.670^{+0.059}_{-0.061}$ \\\\ \n\t\tns & $0.954\\pm 0.042$ \\\\ \n\t\tw0 & $-0.73^{+0.19}_{-0.22}$ \\\\ \n\t\t\\hline\n    \\end{tabular}\n\\end{table}'